## Understanding the EmbeddingQAFinetune object
Loading the prev generated qa object and exploring the structure

**Load previously generated Q and chunks**

In [7]:
from llama_index.finetuning import EmbeddingQAFinetuneDataset

# Load the datasets from saved json files
qa_dataset = EmbeddingQAFinetuneDataset.from_json(
    "data/qa_dataset_25-11-23_100_samples.json"
)

### The QA object:

- **.corpus** # contains a dict where key is a unique ID for each text chunk, and the value is the text chunk (used for generating the Q)
- **.queries** # contains a dict where key is a unique ID for each question, and the value is the question (generated previously)
- **.relevant_docs** # A dict that contains the question ID as the key (prev generated), and as value contains a list of relevant text chunk IDs  (initially list contains only 1 item)
- **.query_docid_pairs** # A list of tuples, where each tuple contains a question str (for some reason not the ID) and a text chunk ID

100 text chunks was used to generate 200 questions

In [2]:
len(qa_dataset.corpus), len(qa_dataset.queries), len(qa_dataset.relevant_docs)

(100, 200, 200)

Inspecting **.corpus**

In [3]:
# return first item from the qa_datset.corpus which is a list of dictionaries
first_corpus_key = next(iter(qa_dataset.corpus))
first_corpus_value = qa_dataset.corpus[first_corpus_key]


print(
    f"First element has: \nUnique ID: {first_corpus_key} \n\nText:\n{first_corpus_value}"
)

First element has: 
Unique ID: 9281784d-734c-4665-b5dd-f6da8fd81859 

Text:
Vejledning om beskæftigelseskravet for ret til barselsdagpenge

1.Indledning
I denne vejledning beskrives hvilke beskæftigelseskrav, der gælder for at opnå ret til barselsdagpenge. Reglerne findes i §§ 27-29 i barselsloven (loven).

I bekendtgørelse nr. 1090 af 1. juni 2021 om opgørelse af beskæftigelseskrav og beregning af barselsdagpenge m.v. (bekendtgørelsen) er fastsat nærmere regler om opgørelsen af beskæftigelseskravet.

Denne vejledning erstatter vejledning nr. 10066 af 29. december 2020 om beskæftigelseskravet for ret til barselsdagpenge.

En lønmodtager har ret til barselsdagpenge, hvis pågældende opfylder ét af de fem krav om tilknytning til arbejdsmarkedet, der er nævnt i lovens § 27, stk. 1.

Kravet om aktuel beskæftigelse dagen før fraværet eller på første fraværsdag samt beskæftigelse i mindst 160 timer indberettet i indkomstregistret inden for de seneste 4 afsluttede kalendermåneder forud for fra

Inspecting **.queries**

In [4]:
# Load the .queries in a similar manner
first_query_key = next(iter(qa_dataset.queries))
first_query_value = qa_dataset.queries[first_query_key]

print(f"First query has: \nUnique ID: {first_query_key} \n\nText:\n{first_query_value}")

First query has: 
Unique ID: 7b4dbc95-9077-45ff-90b0-8dfb1920ce88 

Text:
Hvad er de fem krav om tilknytning til arbejdsmarkedet, som en lønmodtager skal opfylde for at have ret til barselsdagpenge?


Inspecting **.relevant_docs**

In [5]:
# Load the qa_dataset.relevant_docs in a similar manner
first_relevant_docs_key = next(iter(qa_dataset.relevant_docs))
first_relevant_docs_value = qa_dataset.relevant_docs[first_relevant_docs_key]

print(
    f"First question - text pair has : \nQuestion ID: {first_relevant_docs_key} \n\nText ID:\n{first_relevant_docs_value}"
)

First question - text pair has : 
Question ID: 7b4dbc95-9077-45ff-90b0-8dfb1920ce88 

Text ID:
['9281784d-734c-4665-b5dd-f6da8fd81859']


**Since I generated 2 questions per chunk in the previous notebook, there will be 2 questions that refer to the same text ID**

Inspecting **.query_docid_pairs**

In [6]:
qa_dataset.query_docid_pairs[:2]

[('Hvad er de fem krav om tilknytning til arbejdsmarkedet, som en lønmodtager skal opfylde for at have ret til barselsdagpenge?',
  ['9281784d-734c-4665-b5dd-f6da8fd81859']),
 ('Hvad er betingelserne for at opfylde kravet om aktuel beskæftigelse og beskæftigelse i mindst 160 timer for at få ret til barselsdagpenge?',
  ['9281784d-734c-4665-b5dd-f6da8fd81859'])]

## Creatinga joined Q & Chunk dataset

In [ ]:
import pandas as pd
df = pd.DataFrame(list(qa_dataset.relevant_docs.items()),columns = ['question_id','chunk_id'])
df.head()

,question_id,chunk_id
0,7b4dbc95-9077-45ff-90b0-8dfb1920ce88,[9281784d-734c-4665-b5dd-f6da8fd81859]
1,664a0a9c-c46e-429f-bc23-ab529fe39639,[9281784d-734c-4665-b5dd-f6da8fd81859]
2,4a3739b7-d9a0-4e85-9870-18ec4085b53a,[bbeb5fdc-a88b-4981-861c-65d14ff2d3ec]
3,b8603470-549e-462f-be6a-8054c44a4a6d,[bbeb5fdc-a88b-4981-861c-65d14ff2d3ec]
4,3ddfe1da-8a91-444c-8d9b-94fe384de396,[3011dcf7-0ef2-43cb-9668-880ed01a5de5]


In [ ]:
df_corpus = pd.DataFrame(list(qa_dataset.corpus.items()),columns = ['chunk_id','chunk_text'])
df_corpus.head()

,chunk_id,chunk_text
0,9281784d-734c-4665-b5dd-f6da8fd81859,Vejledning om beskæftigelseskravet for ret til...
1,bbeb5fdc-a88b-4981-861c-65d14ff2d3ec,Kravet om at ville have været berettiget til a...
2,3011dcf7-0ef2-43cb-9668-880ed01a5de5,"Det fremgår af lovens § 27, stk. 1, nr. 1, at ..."
3,0ecf96e1-1982-472d-be10-df29d162aafb,Opgørelsen af beskæftigelseskravet sker som ud...
4,4c7f436c-b9b4-49eb-8766-8b111311eef3,Hvis det i forbindelse med Udbetaling Danmarks...


In [ ]:
df_queries = pd.DataFrame(list(qa_dataset.queries.items()),columns = ['question_id','question'])
df_queries.head()

,question_id,question
0,7b4dbc95-9077-45ff-90b0-8dfb1920ce88,Hvad er de fem krav om tilknytning til arbejds...
1,664a0a9c-c46e-429f-bc23-ab529fe39639,Hvad er betingelserne for at opfylde kravet om...
2,4a3739b7-d9a0-4e85-9870-18ec4085b53a,Hvad er kravet for at være berettiget til arbe...
3,b8603470-549e-462f-be6a-8054c44a4a6d,Hvad er kravet for at opfylde beskæftigelseskr...
4,3ddfe1da-8a91-444c-8d9b-94fe384de396,Hvad er kravet for at få ret til barselsdagpen...


In [ ]:
#Duplicate each row of the df_corpus dataframe in order to have a row for each question_id (twice as many)
df_corpus = df_corpus.loc[df_corpus.index.repeat(2)].reset_index(drop=True)
df_corpus.head()

,chunk_id,chunk_text
0,9281784d-734c-4665-b5dd-f6da8fd81859,Vejledning om beskæftigelseskravet for ret til...
1,9281784d-734c-4665-b5dd-f6da8fd81859,Vejledning om beskæftigelseskravet for ret til...
2,bbeb5fdc-a88b-4981-861c-65d14ff2d3ec,Kravet om at ville have været berettiget til a...
3,bbeb5fdc-a88b-4981-861c-65d14ff2d3ec,Kravet om at ville have været berettiget til a...
4,3011dcf7-0ef2-43cb-9668-880ed01a5de5,"Det fremgår af lovens § 27, stk. 1, nr. 1, at ..."


In [ ]:
#Add the df_corpus and the df_queries dataframes together by concatenating them
df_corpus = pd.concat([df_corpus, df_queries], axis=1)
df_corpus.head()

,chunk_id,chunk_text,question_id,question
0,9281784d-734c-4665-b5dd-f6da8fd81859,Vejledning om beskæftigelseskravet for ret til...,7b4dbc95-9077-45ff-90b0-8dfb1920ce88,Hvad er de fem krav om tilknytning til arbejds...
1,9281784d-734c-4665-b5dd-f6da8fd81859,Vejledning om beskæftigelseskravet for ret til...,664a0a9c-c46e-429f-bc23-ab529fe39639,Hvad er betingelserne for at opfylde kravet om...
2,bbeb5fdc-a88b-4981-861c-65d14ff2d3ec,Kravet om at ville have været berettiget til a...,4a3739b7-d9a0-4e85-9870-18ec4085b53a,Hvad er kravet for at være berettiget til arbe...
3,bbeb5fdc-a88b-4981-861c-65d14ff2d3ec,Kravet om at ville have været berettiget til a...,b8603470-549e-462f-be6a-8054c44a4a6d,Hvad er kravet for at opfylde beskæftigelseskr...
4,3011dcf7-0ef2-43cb-9668-880ed01a5de5,"Det fremgår af lovens § 27, stk. 1, nr. 1, at ...",3ddfe1da-8a91-444c-8d9b-94fe384de396,Hvad er kravet for at få ret til barselsdagpen...


In [18]:
#Save the dataframe as a csv file ensure that non ascii characters are encoded as utf-8
df.to_csv('data/chunks_questions-25-11-23_100_samples.csv', encoding='utf-8')